In [1]:
import pandas as pd

In [2]:
## how this data is generated? it is not on  github
df_html = pd.read_parquet("../data/HTML_data.parquet")

In [ ]:
df_html

In [4]:
df = df_html.head(50)

In [5]:
df

,reference_id,domain_start_id,response_url,response_text
0,23884,3292,https://bianchi-industrial.it/products/linear-...,<!doctypehtml><html class=no-js lang=en-GB><me...
1,65092,1265,https://vilagrancha.com/en/1292-complete-clutc...,<!doctypehtml><html lang=en><meta charset=utf-...
2,193015,278,https://www.groupe-lechevalier.com/produit/rou...,<!doctypehtml><html lang=fr xml:lang=fr xmlns:...
3,84150,1187,https://dachnikof.ru/products/Hozblok_Greensto...,"<!doctypehtml><html lang=ru-RU><meta content=""..."
4,72410,3308,https://www.bell.si/Lepila-in-tesnilne-mase/cy...,<!doctypehtml><html lang=sl><meta charset=utf-...
5,193634,181,https://podshipnik-mo.ru/stopornyye-shayby,<!doctypehtml><html lang=ru><head profile=http...
6,43718,1189,https://www.kalydon.com.ar/ubicacion-unica/gba/,<!doctypehtml><html lang=es><meta charset=utf-...
7,98087,1867,https://www.nicotechnika.hu/hu/szigetelt-szers...,<!doctypehtml><html lang=hu-HU><meta charset=u...
8,193619,278,https://www.groupe-lechevalier.com/produit/tra...,<!doctypehtml><html lang=fr xml:lang=fr xmlns:...
9,98901,2378,https://www.lemu.dk/da/catalog/roer-og-fitting...,<!doctypehtml><html class=no-js lang=en xml:la...


In [6]:
import os
import requests
import urllib3
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup

# Suppress SSL warnings globally
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [7]:
def extract_img_attributes(html):
    """
    Extracts attributes from all <img> tags in the provided HTML content.
    Returns a list of dictionaries, each containing attributes of an <img> tag.
    """
    soup = BeautifulSoup(html, 'lxml')
    img_tags = soup.find_all('img')
    img_data = [img.attrs for img in img_tags]  # List comprehension for brevity
    
    return img_data

In [8]:
def save_combined_html(df, output_file="../data/combined.html"):
    # Combine all response text into one HTML file
    with open(output_file, "w", encoding="utf-8") as file:
        for html_content in df["response_text"]:
            file.write(html_content)
            file.write("\n")  # Separate each HTML content by a newline for readability

In [9]:
def download_images_with_local_path(dict_list, base_url, download_folder="../data/images"):
    """
    Downloads images from URLs in img attributes list and stores them locally.
    Adds the local path of each downloaded image back to the dictionary.
    """
    os.makedirs(download_folder, exist_ok=True)
    default_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
    
    for index, img_data in enumerate(dict_list):
        img_url = img_data.get("src")
        
        # Convert relative URLs to absolute URLs
        if img_url and urlparse(img_url).scheme == "":
            img_url = urljoin(base_url, img_url)
            print(f"Converted relative URL to absolute: {img_url}")

        # Replace backslashes with forward slashes
        if img_url:
            img_url = img_url.replace("\\", "/")
        
        # Check if URL is valid and not a data URI
        if img_url and urlparse(img_url).scheme in ["http", "https"]:
            # Generate filename consistent with the original function
            img_name = f"{index}_{os.path.basename(urlparse(img_url).path)}"
            img_path = os.path.join(download_folder, img_name)
            
            # Download the image using requests with SSL verification disabled
            try:
                # Attempt to download with custom User-Agent on the first try
                response = requests.get(img_url, headers=default_headers, stream=True, verify=False)
                response.raise_for_status()  # Raise an error for HTTP issues
                with open(img_path, "wb") as img_file:
                    for chunk in response.iter_content(1024):
                        img_file.write(chunk)
                print(f"Downloaded: {img_name}")
                
                # Add the relative path to the dictionary
                img_data["local_path"] = os.path.relpath(img_path, start=download_folder)
                
            except Exception as e:
                # Print the error and skip to the next item
                print(f"Error downloading {img_url}: {e}")
                
        else:
            # Skip data URIs or invalid URLs
            if img_url and img_url.startswith("data:image"):
                print(f"Skipping item {index} - Data URI found.")
            else:
                print(f"Skipping item {index} - No valid image URL found.")

In [ ]:
# Process each HTML and base URL from the DataFrame
all_images_data = []

for idx, row in df.iterrows():
    html_content = row["response_text"]
    base_url = row["response_url"]
    
    # Extract image attributes for the current HTML page
    img_data = extract_img_attributes(html_content)
    
    # Download images and resolve paths
    download_images_with_local_path(img_data, base_url=base_url)
    
    # Add base URL and image data to the overall list
    for img in img_data:
        img["source_url"] = base_url  # Keep track of the page URL where the image was found
    all_images_data.extend(img_data)

# Optionally, convert all image data to a DataFrame for analysis
img_df = pd.DataFrame(all_images_data)

In [ ]:
def clean_image_folder(folder_path="../data/images"):
    """
    Deletes all files in the specified folder that are not .jpg files or contain 'logo' in the filename (case-insensitive).
    """
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if file is not a .jpg or contains "logo" (case insensitive)
        if not filename.lower().endswith(".jpg") or "logo" in filename.lower():
            try:
                os.remove(file_path)
                print(f"Deleted: {filename}")
            except Exception as e:
                print(f"Error deleting {filename}: {e}")

# Usage
clean_image_folder("../data/images")

In [24]:
import os
import json
from PIL import Image
from tqdm import tqdm
import base64
from dotenv import load_dotenv
import openai

# Load environment variables
assert load_dotenv()

# Set OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')


def create_prompt(categories):
    """
    Creates a comprehensive prompt for the GPT-4 model to analyze images and classify them into specified categories,
    or categorize it independently if no predefined category matches.
    """
    categories_str = ", ".join(categories)
    prompt = f"""
    You are an advanced AI system specializing in image recognition. Your task is to analyze the provided image and determine if it contains any of the following categories: {categories_str}.
    
    **Instructions:**
    - For each category, respond with `true` if the object is clearly visible, and `false` otherwise.
    - If none of the categories match the content of the image, assign your own single-word category to describe the image.
    - Return your analysis in the following JSON format:
    ```json
    {{
        "categories": {{
            "category1": true/false,
            "category2": true/false,
            ...
        }},
        "custom_category": "your-category-here" (if applicable)
    }}
    ```
    - Do not include any explanations or text outside of the JSON object.
    - Ensure the JSON is properly formatted with no syntax errors.
    """
    return prompt.strip()


def preprocess_image(image_path, target_size=(512, 512)):
    """
    Preprocesses an image by resizing and converting it to grayscale.
    """
    try:
        img = Image.open(image_path)
        img = img.resize(target_size, Image.LANCZOS).convert("L")
        return img
    except Exception as e:
        print(f"Error preprocessing image {image_path}: {e}")
        return None

def encode_image_to_base64(image_path):
    """
    Encodes an image to a base64 string.
    """
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        print(f"Error encoding image {image_path} to base64: {e}")
        return None

def call_openai_api(image_path, prompt):
    """
    Calls the OpenAI API to process an image with a given prompt and prints the response.
    """
    try:
        # Encode the image to base64
        encoded_image = encode_image_to_base64(image_path)
        if not encoded_image:
            print(f"Skipping image {image_path} due to encoding error.")
            return

        # Prepare the message
        messages = [
            {"role": "user", "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}}
            ]}
        ]

        # Call the OpenAI API
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )

        # Print the response
        print(f"Response for {image_path}:\n{response.choices[0].message.content}\n")

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")

def main(image_folder, categories):
    """
    Main pipeline for analyzing images with the GPT-4 model.
    """
    prompt = create_prompt(categories)
    import random
    
    image_paths = random.sample(
        [
            os.path.join(image_folder, img)
            for img in os.listdir(image_folder)
            if img.lower().endswith(('.jpg', '.png', '.jpeg'))
        ],
        20
    )

    print(f"Found {len(image_paths)} images to process.")

    # Process each image
    for image_path in tqdm(image_paths, desc="Processing Images"):
        call_openai_api(image_path, prompt)

if __name__ == "__main__":
    # Define folders and categories
    image_folder = "../data/images"
    categories = ["screw", "bolt"]

    print("Starting main process...")
    main(image_folder, categories)

Starting main process...
Found 20 images to process.


Processing Images:   5%|▌         | 1/20 [00:03<01:01,  3.21s/it]

Response for ../data/images/4083_1790000-200x200.jpg:
```json
{
    "categories": {
        "screw": false,
        "bolt": false
    },
    "custom_category": "insecticide"
}
```



Processing Images:  10%|█         | 2/20 [00:05<00:44,  2.47s/it]

Response for ../data/images/24_s_2.jpg:
```json
{
    "categories": {
        "screw": false,
        "bolt": false
    },
    "custom_category": "shed"
}
```



...

Response for ../data/images/452_pre122535.jpg:
```json
{
    "categories": {
        "screw": false,
        "bolt": false
    },
    "custom_category": "drill"
}
```



Processing Images:  95%|█████████▌| 19/20 [00:52<00:02,  2.80s/it]

Response for ../data/images/487_wms100096.jpg:
```json
{
    "categories": {
        "screw": false,
        "bolt": false
    },
    "custom_category": "sanding_disk"
}
```



Processing Images: 100%|██████████| 20/20 [00:55<00:00,  2.76s/it]

Response for ../data/images/33_no-img.jpg:
```json
{
    "categories": {
        "screw": false,
        "bolt": false
    },
    "custom_category": "logo"
}
```

